# DEPENDENCIES

In [ ]:
import IPython.display as display
!pip uninstall -y requests

!pip install -U urllib3 requests
display.clear_output()

In [ ]:
!pip uninstall -y torch torchvision
!pip install torch==2.0.0 torchvision==0.15.1

In [ ]:
!pip install -U openmim
!pip install "mmengine>=0.7.1,<1.0.0" \
 "mmcv>=2.0.0rc4,<2.1.0" \
 "mmdet>=3.0.0,<3.1.0" \
 -f https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html \
--trusted-host download.openmmlab.com

In [ ]:
!git clone https://github.com/open-mmlab/mmyolo.git
%cd mmyolo

!pip install -r requirements/mminstall.txt
# Install albumentations
!pip install -r requirements/albu.txt
# Install MMYOLO
!pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in editable mode,
# thus any local modifications made to the code will take effect without reinstallation.

# Import the dataset to MMYOLO

In [ ]:
#copying the dataset to mmyolo directory

import shutil
import os

# Source directory to copy
src_dir = '/kaggle/input/loserspcb-v2/combined_trialv4_updated'

# Destination directory where the source directory will be copied
dst_dir = '/kaggle/working/mmyolo/datasets'

# Remove destination directory if it exists
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

# Copy the entire directory tree
shutil.copytree(src_dir, dst_dir)

# Modifying the base YOLOv6-s cofig

In [ ]:

config_base_yolov6= """
_base_ = ['../_base_/default_runtime.py']

# ========================Frequently modified parameters======================
# -----data related-----
data_root = './datasets/'  # Root path of data
# Path of train annotation file
train_ann_file = 'train.json'
train_data_prefix = 'train/'  # Prefix of train image path
# Path of val annotation file
val_ann_file = 'val.json'
val_data_prefix = 'val/'  # Prefix of val image path

num_classes = 5  # Number of classes for classification

metainfo = {
    'classes': ('MP','OC','SC','SP','SPC')
    }

train_batch_size_per_gpu = 4
train_num_workers = 1
persistent_workers = True

# -----train val related-----
# Base learning rate for optim_wrapper
base_lr = 0.01
max_epochs = 50  # Maximum training epochs


# ======================= Possible modified parameters =======================
# -----data related-----
img_scale = (1024, 1024)  # width, height
# Dataset type, this will be used to define the dataset
dataset_type = 'YOLOv5CocoDataset'
# Batch size of a single GPU during validation
val_batch_size_per_gpu = 2
# Worker to pre-fetch data for each single GPU during validation
val_num_workers = 1

# Config of batch shapes. Only on val.
# It means not used if batch_shapes_cfg is None.
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# -----model related-----
# The scaling factor that controls the depth of the network structure
deepen_factor = 0.33
# The scaling factor that controls the width of the network structure
widen_factor = 0.5

# -----train val related-----
affine_scale = 0.5  # YOLOv5RandomAffine scaling ratio
lr_factor = 0.01  # Learning rate scaling factor
weight_decay = 0.0005
# Save model checkpoint and validation intervals
save_epoch_intervals = 1
# The maximum checkpoints to keep.
max_keep_ckpts = 1
# Single-scale training is recommended to
# be turned on, which can speed up training.
env_cfg = dict(cudnn_benchmark=True)

# ============================== Unmodified in most cases ===================
model = dict(
    type='YOLODetector',
    data_preprocessor=dict(
        type='YOLOv5DetDataPreprocessor',
        mean=[0., 0., 0.],
        std=[255., 255., 255.],
        bgr_to_rgb=True),
    backbone=dict(
        type='YOLOv6EfficientRep',
        out_indices=[1, 2, 3, 4],
        use_cspsppf=True,
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        norm_cfg=dict(type='BN', momentum=0.03, eps=0.001),
        act_cfg=dict(type='ReLU', inplace=True)),
    neck=dict(
        type='YOLOv6RepBiPAFPN',
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        in_channels=[128, 256, 512, 1024],
        out_channels=[128, 256, 512],
        num_csp_blocks=12,
        norm_cfg=dict(type='BN', momentum=0.03, eps=0.001),
        act_cfg=dict(type='ReLU', inplace=True),
    ),
    bbox_head=dict(
        type='YOLOv6Head',
        head_module=dict(
            type='YOLOv6HeadModule',
            num_classes=num_classes,
            in_channels=[128, 256, 512],
            widen_factor=widen_factor,
            norm_cfg=dict(type='BN', momentum=0.03, eps=0.001),
            act_cfg=dict(type='SiLU', inplace=True),
            featmap_strides=[8, 16, 32]),
        loss_bbox=dict(
            type='IoULoss',
            iou_mode='giou',
            bbox_format='xyxy',
            reduction='mean',
            loss_weight=2.5,
            return_iou=False)),
    train_cfg=dict(
        initial_epoch=4,
        initial_assigner=dict(
            type='BatchATSSAssigner',
            num_classes=num_classes,
            topk=9,
            iou_calculator=dict(type='mmdet.BboxOverlaps2D')),
        assigner=dict(
            type='BatchTaskAlignedAssigner',
            num_classes=num_classes,
            topk=13,
            alpha=1,
            beta=6),
    ),
    test_cfg=dict(
        multi_label=True,
        nms_pre=30000,
        score_thr=0.001,
        nms=dict(type='nms', iou_threshold=0.65),
        max_per_img=300))

# The training pipeline of YOLOv6 is basically the same as YOLOv5.
# The difference is that Mosaic and RandomAffine will be closed in the last 15 epochs. # noqa
pre_transform = [
    dict(type='LoadImageFromFile', backend_args=_base_.backend_args),
    dict(type='LoadAnnotations', with_bbox=True)
]

train_pipeline = [
    *pre_transform,
    dict(type='YOLOv5KeepRatioResize', scale=img_scale),
    dict(
        type='LetterResize',
        scale=img_scale,
        allow_scale_up=True,
        pad_val=dict(img=114.0)),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape'))
]

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    collate_fn=dict(type='yolov5_collate'),
    persistent_workers=persistent_workers,
    pin_memory=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file=train_ann_file,
        data_prefix=dict(img=train_data_prefix),
        filter_cfg=dict(filter_empty_gt=False, min_size=32),
        pipeline=train_pipeline))

test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=_base_.backend_args),
    dict(type='YOLOv5KeepRatioResize', scale=img_scale),
    dict(
        type='LetterResize',
        scale=img_scale,
        allow_scale_up=True,
        pad_val=dict(img=114.0)),
    dict(type='LoadAnnotations', with_bbox=True, _scope_='mmdet'),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor', 'pad_param'))
]

val_dataloader = dict(
    batch_size=val_batch_size_per_gpu,
    num_workers=val_num_workers,
    persistent_workers=persistent_workers,
    pin_memory=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        test_mode=True,
        data_prefix=dict(img=val_data_prefix),
        ann_file=val_ann_file,
        pipeline=test_pipeline,
        batch_shapes_cfg=batch_shapes_cfg))

test_dataloader = val_dataloader

# Optimizer and learning rate scheduler of YOLOv6 are basically the same as YOLOv5. # noqa
# The difference is that the scheduler_type of YOLOv6 is cosine.
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='SGD',
        lr=base_lr,
        momentum=0.937,
        weight_decay=weight_decay,
        nesterov=True,
        batch_size_per_gpu=train_batch_size_per_gpu),
    constructor='YOLOv5OptimizerConstructor')

default_hooks = dict(
    param_scheduler=dict(
        type='YOLOv5ParamSchedulerHook',
        scheduler_type='cosine',
        lr_factor=lr_factor,
        max_epochs=max_epochs),
    logger=dict(type='LoggerHook', interval=5),
    checkpoint=dict(
        type='CheckpointHook',
        interval=save_epoch_intervals,
        max_keep_ckpts=max_keep_ckpts,
        save_best='auto'))

val_evaluator = dict(
    type='mmdet.CocoMetric',
    proposal_nums=(100, 1, 10),
    classwise=True,
    ann_file=data_root + val_ann_file,
    metric='bbox')
test_evaluator = val_evaluator

train_cfg = dict(
    type='EpochBasedTrainLoop',
    max_epochs=max_epochs,
    val_interval=save_epoch_intervals)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')

"""
with open('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_s_syncbn_fast_8xb32-300e_coco.py', 'w') as f:
    f.write(config_base_yolov6)

In [ ]:
#for visualising the config
from mmengine import Config
import json
cfg = Config.fromfile('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_s_syncbn_fast_8xb32-300e_coco.py')
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

print(formatted_cfg)

# yolov6-m config

In [ ]:

config_medium_yolov6 = """

_base_ = './yolov6_v3_s_syncbn_fast_8xb32-300e_coco.py'

# ======================= Possible modified parameters =======================
# -----model related-----
# The scaling factor that controls the depth of the network structure
deepen_factor = 0.6
# The scaling factor that controls the width of the network structure
widen_factor = 0.75

# ============================== Unmodified in most cases =====================
model = dict(
    backbone=dict(
        type='YOLOv6CSPBep',
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        hidden_ratio=2. / 3,
        block_cfg=dict(type='RepVGGBlock'),
        act_cfg=dict(type='ReLU', inplace=True)),
    neck=dict(
        type='YOLOv6CSPRepBiPAFPN',
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        block_cfg=dict(type='RepVGGBlock'),
        hidden_ratio=2. / 3,
        block_act_cfg=dict(type='ReLU', inplace=True)),
    bbox_head=dict(
        type='YOLOv6Head',
        head_module=dict(reg_max=16, widen_factor=widen_factor)))
        
"""
with open('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_m_syncbn_fast_8xb32-300e_coco.py', 'w') as f:
    f.write(config_medium_yolov6)


In [ ]:
#for visualising the config
from mmengine import Config
import json
cfg = Config.fromfile('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_m_syncbn_fast_8xb32-300e_coco.py')
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

print(formatted_cfg)

# yolov6-l 

In [ ]:

config_large_yolov6 = """

_base_ = './yolov6_v3_m_syncbn_fast_8xb32-300e_coco.py'

# ======================= Possible modified parameters =======================
# -----model related-----
# The scaling factor that controls the depth of the network structure
deepen_factor = 1
# The scaling factor that controls the width of the network structure
widen_factor = 1

# ============================== Unmodified in most cases ===================
model = dict(
    backbone=dict(
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        hidden_ratio=1. / 2,
        block_cfg=dict(
            type='ConvWrapper',
            norm_cfg=dict(type='BN', momentum=0.03, eps=0.001)),
        act_cfg=dict(type='SiLU', inplace=True)),
    neck=dict(
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        hidden_ratio=1. / 2,
        block_cfg=dict(
            type='ConvWrapper',
            norm_cfg=dict(type='BN', momentum=0.03, eps=0.001)),
        block_act_cfg=dict(type='SiLU', inplace=True)),
    bbox_head=dict(head_module=dict(widen_factor=widen_factor)))

"""
with open('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_l_syncbn_fast_8xb32-50e_pcb_defect.py', 'w') as f:
    f.write(config_large_yolov6)


In [ ]:
#for visualising the config
from mmengine import Config
import json
cfg = Config.fromfile('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_l_syncbn_fast_8xb32-50e_pcb_defect.py')
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

print(formatted_cfg)

In [ ]:
# saving the config file
from mmengine import Config
import json

# Load the configuration from file
cfg = Config.fromfile('/kaggle/working/mmyolo/configs/yolov6/yolov6_v3_l_syncbn_fast_8xb32-50e_pcb_defect.py')

# Convert the configuration to a dictionary and then to a formatted JSON string
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

# Define the output file path
output_file = '/kaggle/working/formatted_config.json'

# Save the formatted JSON string to a file
with open(output_file, 'w') as f:
    f.write(formatted_cfg)

print(f'Configuration saved as {output_file}')

# Visualizing the dataset 

In [ ]:
# !python tools/analysis_tools/browse_dataset.py configs/yolov6/yolov6_v3_l_syncbn_fast_8xb32-50e_pcb_defect.py --phase train --mode pipeline --out-dir dataset_check_8 --show-number 8 --show-interval 10

# Training model

In [ ]:
!bash ./tools/dist_train.sh configs/yolov6/yolov6_v3_l_syncbn_fast_8xb32-50e_pcb_defect.py 2 --work-dir yolov6_loserspcb_50e/

# Saving the weight file

In [ ]:
# Just to look at the converted data

import shutil

# Directory to be zipped
directory_to_zip = '/kaggle/working/mmyolo/yolov6_loserspcb_50e'

# Destination zip file path
zip_file_path = '/kaggle/working/yolov6_loserspcb_50e'

# Create a zip file
shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)

# Testing the model

In [ ]:
# !python tools/test.py \
# configs/yolov8/yolov8_m_syncbn_fast_8xb16-50e_pcb_defect.py \
# /kaggle/input/inference/best_coco_bbox_mAP_epoch_25.pth\
# --work-dir results/evaluate \
# --out results/results.pkl

# FLOPS 

In [ ]:
# !python tools/analysis_tools/get_flops.py configs/yolov8/yolov8_m_syncbn_fast_8xb16-50e_pcb_defect.py --shape 640 640

# Benchmark

In [ ]:
# !python tools/analysis_tools/benchmark.py configs/yolov8/yolov8_m_syncbn_fast_8xb16-50e_pcb_defect.py \
# /kaggle/input/inference/best_coco_bbox_mAP_epoch_25.pth \
# --repeat-num 5 \
# --max-iter 200 \
# --log-interval 50 \
# --work-dir ./results

# Confusion Matrix

In [ ]:
# code ='''

# import argparse
# import os
# import csv
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.ticker import MultipleLocator
# from mmcv.ops import nms
# from mmengine import Config, DictAction
# from mmengine.fileio import load
# from mmengine.registry import init_default_scope
# from mmengine.utils import ProgressBar

# from mmdet.evaluation import bbox_overlaps
# from mmdet.registry import DATASETS
# from mmdet.utils import replace_cfg_vals, update_data_root


# def parse_args():
#     parser = argparse.ArgumentParser(
#         description='Generate confusion matrix from detection results')
#     parser.add_argument('config', help='test config file path')
#     parser.add_argument(
#         'prediction_path', help='prediction path where test .pkl result')
#     parser.add_argument(
#         'save_dir', help='directory where confusion matrix will be saved')
#     parser.add_argument(
#         '--show', action='store_true', help='show confusion matrix')
#     parser.add_argument(
#         '--color-theme',
#         default='plasma',
#         help='theme of the matrix color map')
#     parser.add_argument(
#         '--score-thr',
#         type=float,
#         default=0.3,
#         help='score threshold to filter detection bboxes')
#     parser.add_argument(
#         '--tp-iou-thr',
#         type=float,
#         default=0.5,
#         help='IoU threshold to be considered as matched')
#     parser.add_argument(
#         '--nms-iou-thr',
#         type=float,
#         default=None,
#         help='nms IoU threshold, only applied when users want to change the'
#         'nms IoU threshold.')
#     parser.add_argument(
#         '--cfg-options',
#         nargs='+',
#         action=DictAction,
#         help='override some settings in the used config, the key-value pair '
#         'in xxx=yyy format will be merged into config file. If the value to '
#         'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
#         'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
#         'Note that the quotation marks are necessary and that no white space '
#         'is allowed.')
#     args = parser.parse_args()
#     return args


# def calculate_confusion_matrix(dataset,
#                                results,
#                                score_thr=0,
#                                nms_iou_thr=None,
#                                tp_iou_thr=0.5):
#     """Calculate the confusion matrix.

#     Args:
#         dataset (Dataset): Test or val dataset.
#         results (list[ndarray]): A list of detection results in each image.
#         score_thr (float|optional): Score threshold to filter bboxes.
#             Default: 0.
#         nms_iou_thr (float|optional): nms IoU threshold, the detection results
#             have done nms in the detector, only applied when users want to
#             change the nms IoU threshold. Default: None.
#         tp_iou_thr (float|optional): IoU threshold to be considered as matched.
#             Default: 0.5.
#     """
#     num_classes = len(dataset.metainfo['classes'])
#     confusion_matrix = np.zeros(shape=[num_classes + 1, num_classes + 1])
#     assert len(dataset) == len(results)
#     prog_bar = ProgressBar(len(results))
#     for idx, per_img_res in enumerate(results):
#         res_bboxes = per_img_res['pred_instances']
#         gts = dataset.get_data_info(idx)['instances']
#         analyze_per_img_dets(confusion_matrix, gts, res_bboxes, score_thr,
#                              tp_iou_thr, nms_iou_thr)
#         prog_bar.update()
#     return confusion_matrix


# def analyze_per_img_dets(confusion_matrix,
#                          gts,
#                          result,
#                          score_thr=0,
#                          tp_iou_thr=0.5,
#                          nms_iou_thr=None):
#     """Analyze detection results on each image.

#     Args:
#         confusion_matrix (ndarray): The confusion matrix,
#             has shape (num_classes + 1, num_classes + 1).
#         gt_bboxes (ndarray): Ground truth bboxes, has shape (num_gt, 4).
#         gt_labels (ndarray): Ground truth labels, has shape (num_gt).
#         result (ndarray): Detection results, has shape
#             (num_classes, num_bboxes, 5).
#         score_thr (float): Score threshold to filter bboxes.
#             Default: 0.
#         tp_iou_thr (float): IoU threshold to be considered as matched.
#             Default: 0.5.
#         nms_iou_thr (float|optional): nms IoU threshold, the detection results
#             have done nms in the detector, only applied when users want to
#             change the nms IoU threshold. Default: None.
#     """
#     true_positives = np.zeros(len(gts))
#     gt_bboxes = []
#     gt_labels = []
#     for gt in gts:
#         gt_bboxes.append(gt['bbox'])
#         gt_labels.append(gt['bbox_label'])

#     gt_bboxes = np.array(gt_bboxes)
#     gt_labels = np.array(gt_labels)

#     unique_label = np.unique(result['labels'].numpy())

#     for det_label in unique_label:
#         mask = (result['labels'] == det_label)
#         det_bboxes = result['bboxes'][mask].numpy()
#         det_scores = result['scores'][mask].numpy()

#         if nms_iou_thr:
#             det_bboxes, _ = nms(
#                 det_bboxes, det_scores, nms_iou_thr, score_threshold=score_thr)
#         ious = bbox_overlaps(det_bboxes[:, :4], gt_bboxes)
#         for i, score in enumerate(det_scores):
#             det_match = 0
#             if score >= score_thr:
#                 for j, gt_label in enumerate(gt_labels):
#                     if ious[i, j] >= tp_iou_thr:
#                         det_match += 1
#                         if gt_label == det_label:
#                             true_positives[j] += 1  # TP
#                         confusion_matrix[gt_label, det_label] += 1
#                 if det_match == 0:  # BG FP
#                     confusion_matrix[-1, det_label] += 1
#     for num_tp, gt_label in zip(true_positives, gt_labels):
#         if num_tp == 0:  # FN
#             confusion_matrix[gt_label, -1] += 1


# def save_confusion_matrix(confusion_matrix, labels, save_dir):
#     """Save confusion matrix to a CSV file.

#     Args:
#         confusion_matrix (ndarray): The confusion matrix.
#         labels (list[str]): List of class names.
#         save_dir (str): Directory where the confusion matrix will be saved.
#     """
#     save_path = os.path.join(save_dir, 'confusion_matrix.csv')
#     with open(save_path, 'w', newline='') as csvfile:
#         csvwriter = csv.writer(csvfile)
#         csvwriter.writerow([''] + labels + ['background'])
#         for i, row in enumerate(confusion_matrix):
#             csvwriter.writerow([labels[i] if i < len(labels) else 'background'] + row.tolist())


# def plot_confusion_matrix(confusion_matrix,
#                           labels,
#                           save_dir=None,
#                           show=True,
#                           title='Normalized Confusion Matrix',
#                           color_theme='plasma'):
#     """Draw confusion matrix with matplotlib.

#     Args:
#         confusion_matrix (ndarray): The confusion matrix.
#         labels (list[str]): List of class names.
#         save_dir (str|optional): If set, save the confusion matrix plot to the
#             given path. Default: None.
#         show (bool): Whether to show the plot. Default: True.
#         title (str): Title of the plot. Default: `Normalized Confusion Matrix`.
#         color_theme (str): Theme of the matrix color map. Default: `plasma`.
#     """
#     # normalize the confusion matrix
#     per_label_sums = confusion_matrix.sum(axis=1)[:, np.newaxis]
#     confusion_matrix = \
#         confusion_matrix.astype(np.float32) / per_label_sums * 100

#     num_classes = len(labels)
#     fig, ax = plt.subplots(
#         figsize=(0.5 * num_classes, 0.5 * num_classes * 0.8), dpi=180)
#     cmap = plt.get_cmap(color_theme)
#     im = ax.imshow(confusion_matrix, cmap=cmap)
#     plt.colorbar(mappable=im, ax=ax)

#     title_font = {'weight': 'bold', 'size': 12}
#     ax.set_title(title, fontdict=title_font)
#     label_font = {'size': 10}
#     plt.ylabel('Ground Truth Label', fontdict=label_font)
#     plt.xlabel('Prediction Label', fontdict=label_font)

#     # draw locator
#     xmajor_locator = MultipleLocator(1)
#     xminor_locator = MultipleLocator(0.5)
#     ax.xaxis.set_major_locator(xmajor_locator)
#     ax.xaxis.set_minor_locator(xminor_locator)
#     ymajor_locator = MultipleLocator(1)
#     yminor_locator = MultipleLocator(0.5)
#     ax.yaxis.set_major_locator(ymajor_locator)
#     ax.yaxis.set_minor_locator(yminor_locator)

#     # draw grid
#     ax.grid(True, which='minor', linestyle='-')

#     # draw label
#     ax.set_xticks(np.arange(num_classes))
#     ax.set_yticks(np.arange(num_classes))
#     ax.set_xticklabels(labels)
#     ax.set_yticklabels(labels)

#     ax.tick_params(
#         axis='x', bottom=False, top=True, labelbottom=False, labeltop=True)
#     plt.setp(
#         ax.get_xticklabels(), rotation=45, ha='left', rotation_mode='anchor')

#     # draw confution matrix value
#     for i in range(num_classes):
#         for j in range(num_classes):
#             ax.text(
#                 j,
#                 i,
#                 '{}%'.format(
#                     int(confusion_matrix[
#                         i,
#                         j]) if not np.isnan(confusion_matrix[i, j]) else -1),
#                 ha='center',
#                 va='center',
#                 color='w',
#                 size=7)

#     ax.set_ylim(len(confusion_matrix) - 0.5, -0.5)  # matplotlib>3.1.1

#     fig.tight_layout()
#     if save_dir is not None:
#         plt.savefig(
#             os.path.join(save_dir, 'confusion_matrix.png'), format='png')
#     if show:
#         plt.show()


# def main():
#     args = parse_args()

#     cfg = Config.fromfile(args.config)

#     # replace the ${key} with the value of cfg.key
#     cfg = replace_cfg_vals(cfg)

#     # update data root according to MMDET_DATASETS
#     update_data_root(cfg)

#     if args.cfg_options is not None:
#         cfg.merge_from_dict(args.cfg_options)

#     init_default_scope(cfg.get('default_scope', 'mmdet'))

#     results = load(args.prediction_path)

#     if not os.path.exists(args.save_dir):
#         os.makedirs(args.save_dir)

#     dataset = DATASETS.build(cfg.test_dataloader.dataset)

#     confusion_matrix = calculate_confusion_matrix(dataset, results,
#                                                   args.score_thr,
#                                                   args.nms_iou_thr,
#                                                   args.tp_iou_thr)
#     class_labels = list(dataset.metainfo['classes']) + ['background']
#     save_confusion_matrix(confusion_matrix, class_labels, args.save_dir)
#     plot_confusion_matrix(
#         confusion_matrix,
#         class_labels,
#         save_dir=args.save_dir,
#         show=args.show,
#         color_theme=args.color_theme)


# if __name__ == '__main__':
#     main()
# '''

# with open('./tools/analysis_tools/confusion_matrix_2.py', 'w') as f:
#     f.write(code)

In [ ]:
# !python tools/analysis_tools/confusion_matrix_2.py \
# configs/yolov8/yolov8_m_syncbn_fast_8xb16-50e_pcb_defect.py  \
# results/results.pkl  \
# ./results \
# --show \
# --score-thr 0.5 \
# --tp-iou-thr 0.5

# Saving the Results

In [ ]:
# import shutil

# # Directory to be zipped
# directory_to_zip = '/kaggle/working/mmyolo/results'

# # Destination zip file path
# zip_file_path = '/kaggle/working/yolov8_dspcbsd_results.zip'

# # Create a zip file
# shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)

# Inference

In [ ]:
# from mmdet.apis import init_detector, inference_detector

# config = 'configs/yolov8/yolov8_m_syncbn_fast_8xb16-50e_pcb_defect.py'
# checkpoint = '/kaggle/input/inference/best_coco_bbox_mAP_epoch_25.pth'

# model = init_detector(config, checkpoint, device='cuda:0')  # 'cpu' or device='cuda:0'

# img = '/kaggle/input/dspcbsd/Data_COCO/val/0066079.jpg'

# result = inference_detector(model, img)

In [ ]:
# from mmdet.apis import DetInferencer
# import glob

# # Choose to use a config
# config = '/content/mmdetection/configs/atss/atss_r50_fpn_1x_raccoon.py'

# # Setup a checkpoint file to load from work_dirs folder
# checkpoint = glob.glob('/content/mmdetection/work_dirs/atss_r50_fpn_1x_raccoon/best_coco_bbox_mAP_epoch_1.pth')[0]

# # Set the device to be used for evaluation
# device = 'cuda:0'

# # Initialize the DetInferencer
# inferencer = DetInferencer(config, checkpoint, device)

# # Use the detector to do inference
# img = '/content/mmdetection/Raccoons-2/test/raccoon-66_jpg.rf.f0160e8a2f9a23455b0394df3607859f.jpg'
# result = inferencer(img, out_dir='./output')